<a href="https://colab.research.google.com/github/xoxominji/22-1-ESAA-Practice/blob/main/%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [6]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
xrd_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [9]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(random_state=42)

In [10]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(random_state=42)

In [15]:
from sklearn.metrics import accuracy_score 
for clf in (rnd_clf, xrd_clf, svm_clf, mlp_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.969
ExtraTreesClassifier 0.973


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC 0.8782857142857143
MLPClassifier 0.9647142857142857


1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [12]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('xrf', xrd_clf), ('svm', svm_clf), ('mlp',mlp_clf)],
    voting='hard'
)

In [16]:
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


0.9731428571428572


> 개별 모델의 성능보다 높은 것을 알 수 있다

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [17]:
voting_clf_new = VotingClassifier(
    estimators=[('rf', rnd_clf), ('xrf', xrd_clf), ('mlp',mlp_clf)],
    voting='hard'
)

In [18]:
voting_clf_new.fit(X_train, y_train)
y_pred = voting_clf_new.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9738571428571429


> svm을 포함하고 있는 모델보다 성능이 미세하지만 증가했다

1-d. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [19]:
voting_clf_soft = VotingClassifier(
    estimators=[('rf', rnd_clf), ('xrf', xrd_clf), ('mlp',mlp_clf)],
    voting='soft'
)

voting_clf_soft.fit(X_train, y_train)
y_pred = voting_clf_soft.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9715714285714285
